https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_adaptive_rag_local/  

https://www.youtube.com/watch?v=bq1Plo2RhYI  


In [23]:
from langchain_ollama import ChatOllama

llm_model = "llama3.2:3b-instruct-fp16"

llm_json_mode = ChatOllama(model=llm_model, temperature=0, format='json')

In [24]:
from langchain_nomic.embeddings import NomicEmbeddings

embeddings = NomicEmbeddings(model='nomic-embed-text-v1.5', inference_mode='local')

In [ ]:
from langchain.schema import Document
import json

def prepare_qa_documents(file_path):
    with open(file_path, 'r') as f:
        qa_data = json.load(f)
    
    documents = [
        Document(
            page_content= f"Q: {item['question']} A: {item['answer']}",
            metadata={"question": item["question"]}
        )
        for item in qa_data
    ]
    
    return documents

documents = prepare_qa_documents("../data/home0001qa.json")
print(documents[:1])

In [26]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents, embeddings)

retriever = vectorstore.as_retriever(k=3)

In [ ]:
retriever.invoke("Do i own my 0001 home outright?")

In [28]:
# Doc grader instructions
doc_grader_instructions = """You are a grader assessing relevance of a retrieved document to a user question.

If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant."""

# Grader prompt
doc_grader_prompt = """Here is the retrieved document: \n\n {document} \n\n Here is the user question: \n\n {question}. 

This carefully and objectively assess whether the document contains at least some information that is relevant to the question.

Return JSON with single key, binary_score, that is 'yes' or 'no' score to indicate whether the document contains at least some information that is relevant to the question."""

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

# Test
question = "Does furniture come included?"
docs = retriever.invoke(question)
doc_txt = docs[0].page_content
doc_grader_prompt_formatted = doc_grader_prompt.format(
    document=doc_txt, question=question
)
result = llm_json_mode.invoke(
    [SystemMessage(content=doc_grader_instructions)]
    + [HumanMessage(content=doc_grader_prompt_formatted)]
)

print(doc_txt)
print(json.loads(result.content))

In [34]:
def grade_documents(documents, question):
  
    # Score each doc
    filtered_docs = []

    for d in documents:
        doc_grader_prompt_formatted = doc_grader_prompt.format(
            document=d.page_content, question=question
        )
        result = llm_json_mode.invoke(
            [SystemMessage(content=doc_grader_instructions)]
            + [HumanMessage(content=doc_grader_prompt_formatted)]
        )
        grade = json.loads(result.content)["binary_score"]
        # Document relevant
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        # Document not relevant
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            # We do not include the document in filtered_docs
            continue

    return {"documents": filtered_docs}

In [ ]:
question = "Do i own my 0001 home outright?"
retrieved_docs = retriever.invoke(question)
filtered_docs = grade_documents(retrieved_docs, question)


In [ ]:
test_string = """ 
    this is a {test}
"""

formatted_test_string = test_string.format(test="hello")

print(formatted_test_string)

In [32]:
# TO DO: HALLUCINATION GRADER
# ANSWER GRADER 